In [8]:
# ╭────────────────────────────  install once  ───────────────────────────╮
# %pip install -q ccxt pandas numpy matplotlib ta
# ╰───────────────────────────────────────────────────────────────────────╯
from typing import Optional
import pandas as pd, numpy as np, ccxt, matplotlib.pyplot as plt, importlib
import break_retest_alert as bot         
importlib.reload(bot)

# ───── CONFIG (same as live bot) ─────────────────────────────────────────
PAIR            = "SOL/USDT"
TF              = "15m"
LOOKBACK_DAYS   = 180

EMA_FAST, EMA_MID, EMA_SLOW = 7, 14, 28
ADX_LEN     = 14
ADX_THRES   = 10
VOL_MULT    = 1.05
MIN_GAP_ATR = 0.08

ATR_SL_MULT = 1.0
ATR_TP_MULT = 3.0
BE_AFTER_ATR= 1.0
TRAIL_AFTER_R = 2.0            # begin trailing after +2 R
TRAIL_ATR     = 1.0            # trail distance

RISK_PCT   = 0.02              # risk % of equity per trade
START_EQ   = 1_000.0
HIGHER_TF_BIAS = False         # flip True to require EMA50>EMA200

# ───── DOWNLOAD DATA ────────────────────────────────────────────────────
ex = ccxt.bybit()
since = ex.milliseconds() - LOOKBACK_DAYS*86_400_000
rows  = []
while since < ex.milliseconds():
    chunk = ex.fetch_ohlcv(PAIR, TF, since=since, limit=1000)
    if not chunk: break
    rows += chunk;  since = chunk[-1][0] + 1

df = (pd.DataFrame(rows, columns="ts o h l c v".split())
        .assign(ts=lambda d: pd.to_datetime(d.ts, unit="ms", utc=True))
        .set_index("ts").astype(float))


# ── 2. indicators – reuse live helper ───────────────────────────────────────
bot.add_indicators(df)

# ───── BACK-TEST LOOP ──────────────────────────────────────────────────
equity, eq_curve = START_EQ, []
pos = None
trades = []

for i in range(max(EMA_SLOW, 30), len(df)):
    bar = df.iloc[i]

    # manage open
    if pos:
        # stop-loss / take-profit
        hit_sl = bar.l <= pos["sl"] <= bar.h
        hit_tp = bar.l <= pos["tp"] <= bar.h
        if hit_sl or hit_tp:
            pnl = -pos["risk"] if hit_sl else pos["risk"]*ATR_TP_MULT/(0.5 if pos["half"] else 1)
            equity += pnl
            trades.append({"exit":bar.name, "pnl":pnl, "type":"SL" if hit_sl else "TP"})
            pos = None
        else:
            # BE lock
            if (not pos["be"]) and \
               ((bar.c >= pos["entry"]+BE_AFTER_ATR*bar.atr and pos["side"]=="long") or
                (bar.c <= pos["entry"]-BE_AFTER_ATR*bar.atr and pos["side"]=="short")):
                pos["sl"] = pos["entry"]; pos["be"]=True
            # trailing ATR after +2 R
            if pos["be"]:
                two_r = pos["entry"] + TRAIL_AFTER_R*pos["risk_per_unit"] if pos["side"]=="long" \
                        else pos["entry"] - TRAIL_AFTER_R*pos["risk_per_unit"]
                if (pos["side"]=="long" and bar.c>=two_r):
                    pos["sl"] = max(pos["sl"], bar.c - TRAIL_ATR*bar.atr)
                elif (pos["side"]=="short" and bar.c<=two_r):
                    pos["sl"] = min(pos["sl"], bar.c + TRAIL_ATR*bar.atr)

            # +1 R partial
            if (not pos["half"]) and \
               ((bar.c - pos["entry"] >= pos["risk_per_unit"] and pos["side"]=="long") or
                (pos["entry"] - bar.c >= pos["risk_per_unit"] and pos["side"]=="short")):
                equity += 0.5 * pos["risk"]
                pos["risk"] *= 0.5; pos["half"]=True

    # new entry
    if pos is None:
        sig = bot.qualified_signal(df.iloc[:i+1])
        if sig:
            side  = "long" if sig=="bull" else "short"
            entry = bar.c
            stop  = entry - ATR_SL_MULT*bar.atr if side=="long" else entry + ATR_SL_MULT*bar.atr
            risk  = equity * RISK_PCT
            pos = dict(side=side, entry=entry, sl=stop,
                       tp=entry+ATR_TP_MULT*bar.atr if side=="long" else entry-ATR_TP_MULT*bar.atr,
                       risk=risk, risk_per_unit=abs(entry-stop),
                       half=False, be=False)

    eq_curve.append(equity)

# ───── RESULTS ─────────────────────────────────────────────────────────
series = pd.Series(eq_curve, index=df.index[-len(eq_curve):])
t_df   = pd.DataFrame(trades)
win    = (t_df.type=="TP").mean()*100 if not t_df.empty else 0
max_dd = (series.min()-START_EQ)/START_EQ*100

print(f"Trades       : {len(t_df)}")
print(f"Win-rate     : {win:.1f}%")
print(f"Final equity : ${equity:,.0f}")
print(f"Max draw-down: {max_dd:.1f}%")
if not t_df.empty:
    expectancy = t_df.pnl.sum()/abs(t_df.pnl).sum()
    print(f"Expectancy R : {expectancy:.2f}")

series.plot(title="Equity Curve – 15-min Triple-EMA", figsize=(10,4), grid=True)
plt.ylabel("USDT"); plt.show()
print(t_df.tail(15))


AttributeError: 'DataFrame' object has no attribute 'close'